In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools
import random
import helpers 
from markov import MarkovModel

pd.set_option("display.max_columns", None)

In [3]:
all_painted = pd.read_csv("all_painted.csv").drop("Unnamed: 0", axis=1)

In [4]:
# 4 corners X 4 corner locations x 9 drip locations = 144 unique drips x 17 second options = 2448 drip + time options
# Lots of options
# Too many options
# Break into parts

# Pick a corner. Itterate through them all?
# Given a corner + corner location where should the drip be? Markov?
# Given a drip how long should it drip for? Select from distribution of best paintings. 
# Given a drip location (maybe current corner?) where should the next corner location be?



In [5]:
corner_drip_scored_dict = helpers.create_corner_drip_scored_dict(all_painted)

drip_corner_scored_dict = helpers.create_drip_corner_scored_dict(all_painted)

drips_with_scores = pd.concat([ helpers.select_one_corner(i, all_painted) for i in range(5)])
drips_with_scores = drips_with_scores[drips_with_scores['drip_x'] != 0].reset_index()

top_score_times = drips_with_scores[drips_with_scores['hanna_second_score'] == 2]['time'].tolist()
top_score_number_drips = all_painted[all_painted['hanna_second_score'] == 2]['number_drips'].tolist()

In [6]:
papers = [
    {"number_drips": top_score_number_drips[random.randint(0, 26)], "corner_1": corner}
    for corner in corner_drip_scored_dict.keys()
]

for paper in papers:
    corner = paper['corner_1']
    number_drips = paper['number_drips']
    paper['drip_1'] = max(corner_drip_scored_dict[corner], key=corner_drip_scored_dict[corner].get)
    paper['time_1'] = top_score_times[random.randint(0, 110)]
    for drip_number in range(1, number_drips+1):
        paper[f'corner_{drip_number + 1}'] = max(drip_corner_scored_dict[paper[f'drip_{drip_number}']], key=drip_corner_scored_dict[paper[f'drip_{drip_number}']].get) 
        paper[f'drip_{drip_number + 1}'] = max(corner_drip_scored_dict[paper[f'corner_{drip_number + 1}']], key=corner_drip_scored_dict[paper[f'corner_{drip_number + 1}']].get)
        paper[f'time_{drip_number + 1}'] = top_score_times[random.randint(0, 110)]

In [8]:
generate_papers = pd.DataFrame(papers)

In [ ]:
generate_papers.to_csv("run_6.csv")

In [ ]:
# This was too deterministic and lead to a lot of loops
# Going to add a little more random chance in theres

In [80]:
mm = MarkovModel(all_painted)

/Users/hanna/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/hanna/.pyenv/versions/anaconda3-2018.12/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [82]:
papers_2 = [
    {"number_drips": mm.get_number_drips()}
    for i in range(16)
]

for paper in papers_2:
    number_drips = paper['number_drips']

    for drip_number in range(1, number_drips+1):
        if drip_number == 1:
            paper[f'corner_{drip_number}'] = mm.get_corner()
        else:
            paper[f'corner_{drip_number}'] = mm.get_corner_given_drip(paper[f'drip_{drip_number - 1}'] )
        paper[f'drip_{drip_number}'] = mm.get_drip_given_corner(paper[f'corner_{drip_number}'])
        paper[f'time_{drip_number}'] = mm.get_time()



In [85]:
generate_papers_2 = pd.DataFrame(papers_2) 
generate_papers_2.to_csv("run_7.csv")